In [ ]:
!pip install tensorflow pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/Shareddrives/Motif_Batik_Rani_Riau'

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import numpy as np
import imgaug.augmenters as iaa
from tqdm import tqdm

In [ ]:
data_dir = '/content/drive/Shareddrives/Motif_Batik_Rani_Riau'
categories = os.listdir(data_dir)
img_size = 224

def load_images(data_dir, categories, img_size):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = Image.open(os.path.join(path, img)).convert("RGB")
                img_array = img_array.resize((img_size, img_size))
                data.append(np.array(img_array))
                labels.append(class_num)
            except Exception as e:
                print(e)
    return np.array(data), np.array(labels)

data, labels = load_images(data_dir, categories, img_size)

# Normalize data
data = data / 255.0

In [ ]:
import os
from PIL import Image
import numpy as np
import imgaug.augmenters as iaa
from tqdm import tqdm

# Definisikan folder output
output_dir = '/content/drive/Shareddrives/Motif_Batik_Rani_Riau/augmented'
os.makedirs(output_dir, exist_ok=True)

# Mendefinisikan direktori dataset
data_dir = '/content/drive/Shareddrives/Motif_Batik_Rani_Riau'
img_size = 224
images_per_image = 21  # Jumlah augmentasi yang ingin Anda terapkan

# Mendefinisikan augmentasi yang akan dilakukan
augmentations = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flips
    iaa.Affine(rotate=(-20, 20)),  # random rotations
    iaa.Multiply((0.8, 1.2)),  # change brightness
    iaa.Crop(percent=(0, 0.2)),  # random cropping
    iaa.LinearContrast((0.75, 1.5)),  # change contrast
    iaa.Affine(scale=(0.8, 1.2)),  # zoom
])

def load_images(data_dir, img_size):
    """
    Memuat gambar dari direktori dan mengubah ukurannya.
    """
    categories = os.listdir(data_dir)
    for category in categories:
        path = os.path.join(data_dir, category)
        if os.path.isdir(path):
            class_num = categories.index(category)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                if os.path.isfile(img_path):
                    try:
                        img = Image.open(img_path).convert("RGB")
                        img = img.resize((img_size, img_size))
                        yield np.array(img, dtype=np.uint8), class_num
                    except Exception as e:
                        print(f"Error processing {img_path}: {e}")

def save_augmented_image(image, label, idx, categories, directory):
    """
    Menyimpan satu gambar augmented ke disk.
    """
    img = Image.fromarray(image)
    img.save(os.path.join(directory, f"augmented_{idx}_label_{categories[label]}.png"))

# Mendapatkan kategori gambar
categories = os.listdir(data_dir)

# Memuat gambar awal dan menghitung total gambar
images = list(load_images(data_dir, img_size))
total_images = len(images)

# Menghitung total gambar setelah augmentasi
total_augmented_images = total_images * images_per_image

# Menggunakan tqdm untuk progress bar pada proses augmentasi dan penyimpanan
with tqdm(total=total_augmented_images, desc="Augmenting and saving images") as pbar:
    for idx, (img, label) in enumerate(images):
        img = np.array(img, dtype=np.uint8)  # Pastikan gambar dalam format uint8
        for i in range(images_per_image):
            augmented_image = augmentations(image=img)
            save_augmented_image(augmented_image, label, idx * images_per_image + i, categories, output_dir)
            pbar.update(1)  # Update progress bar

print(f"Jumlah total gambar setelah augmentasi: {total_augmented_images}")


Augmenting and saving images: 100%|██████████| 2268/2268 [01:27<00:00, 25.90it/s]

Jumlah total gambar setelah augmentasi: 2268
